In [1]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix, csc_matrix, random
from sklearn.decomposition import TruncatedSVD
from scipy import sparse
import time
import tqdm

def generate_random_csr(n_rows, n_cols, density=0.2):
    sparse_matrix = sparse.random(n_rows, n_cols, density=density, format='csr')
    nnz = sparse_matrix.indptr[-1]
    sparse_matrix.data = np.random.randint(128, size=nnz)
    return sparse_matrix

def sparse_random_matrix(m, n, density=0.1):
    A = random(m, n, density=density, format='csr')
    nnz = A.indptr[-1]
    A.data = np.random.randn(nnz)
    return A

def soft_threshold(x, lambda_):
    return np.sign(x) * np.maximum(np.abs(x) - lambda_, 0)

def robust_pca(D, lambda_, tol, maxIter):
    m, n = D.shape
    Y = D.copy()
    L = np.zeros_like(D)
    S = np.zeros_like(D)
    mu = 1e-3  # TODO: tune
    rho = 1.5  # TODO: tune
    error = tol + 1
    k = 0

    # Perform ADMM iterations
    while k < maxIter and error > tol:
        # Update L
        U, Sigma, VT = svds(Y - S + Y/mu, k=min(m, n)-1)
        Sigma_thresh = np.diag(np.maximum(Sigma - 1/mu, 0))
        L = U @ Sigma_thresh @ VT

        S = soft_threshold(Y - L + Y/mu, lambda_/mu)

        Y += mu * rho * (D - L - S)

        error = np.linalg.norm(D - L - S, 'fro') / np.linalg.norm(D, 'fro')

        k += 1

    return L, S

# synthetic data set for testing
m, n = 100, 50  # dimensions of the matrix
rank = 5        # rank of the low-rank component
sparsity = 0.1  # sparsity level of the sparse component

# low-rank matrix
U = np.random.randn(m, rank)
V = np.random.randn(rank, n)
L_true = U @ V

# sparse matrix
S_true = sparse_random_matrix(m, n, density=sparsity).A

# data matrix
D = L_true + S_true

# params (for Robust PCA)
lambda_ = 1 / np.sqrt(max(m, n))
tol = 1e-5
maxIter = 1000

L, S = robust_pca(D, lambda_, tol, maxIter)

print(f"Rank of recovered low-rank matrix: {np.linalg.matrix_rank(L)}")
print(f"Sparsity level of recovered sparse matrix: {np.count_nonzero(S) / (m*n)}")
print("Done with Robust PCA simulation")

Rank of recovered low-rank matrix: 5
Sparsity level of recovered sparse matrix: 0.116
Done with Robust PCA simulation


In [4]:
%pip install cvxpy

     |████████████████████████████████| 1.1 MB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 227 kB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 987 kB 30.6 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 252 kB 42.5 MB/s eta 0:00:01
     |████████████████████████████████| 105 kB 37.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import cvxpy as cp

# synthetic data set for testing
m, n = 100, 50  # dimensions of the matrix
rank = 5        # rank of the low-rank component
sparsity = 0.1  # sparsity level of the sparse component

# low-rank matrix
U = np.random.randn(m, rank)
V = np.random.randn(rank, n)
L_true = U @ V

# rnd sparse matrix
S_true = sparse_random_matrix(m, n, density=sparsity).A

# data matrix
D = L_true + S_true

L = cp.Variable((m, n))
S = cp.Variable((m, n))
lambda_ = 1 / np.sqrt(max(m, n))

# Robust PCA
problem = cp.Problem(cp.Minimize(cp.normNuc(L) + lambda_ * cp.norm1(S)),
                     [D == L + S])

problem.solve(solver=cp.SCS)

# low-rank
L_optimal = L.value
# sparse
S_optimal = S.value

print("Rank of the recovered low-rank matrix:", np.linalg.matrix_rank(L_optimal))
print("Sparsity level of the recovered sparse matrix:", np.count_nonzero(S_optimal) / (m * n))

Rank of the recovered low-rank matrix: 50
Sparsity level of the recovered sparse matrix: 1.0


In [8]:
import cvxpy as cp

epsilon = 1e-5

# synthetic data set for testing
m, n = 100, 50  # dimensions of the matrix
rank = 5        # rank of the low-rank component
sparsity = 0.1  # sparsity level of the sparse component

# low-rank matrix
U = np.random.randn(m, rank)
V = np.random.randn(rank, n)
L_true = U @ V

# rnd sparse matrix
S_true = sparse_random_matrix(m, n, density=sparsity).A

# data matrix
D = L_true + S_true

L = cp.Variable((m, n))
S = cp.Variable((m, n))
lambda_ = 1 / np.sqrt(max(m, n))

# Robust PCA
problem = cp.Problem(cp.Minimize(cp.normNuc(L) + lambda_ * cp.norm1(S)),
                     [cp.norm(D - L - S, "fro") <= epsilon])

problem.solve(solver=cp.SCS)

# low-rank
L_optimal = L.value
# sparse
S_optimal = S.value

print("Rank of the recovered low-rank matrix:", np.linalg.matrix_rank(L_optimal))
print("Sparsity level of the recovered sparse matrix:", np.count_nonzero(S_optimal) / (m * n))

Rank of the recovered low-rank matrix: 50
Sparsity level of the recovered sparse matrix: 1.0
